In [ ]:
# PGD for PanopticFPN (MMDetection 3.x)
import os, copy, torch, numpy as np
from PIL import Image
from mmengine.config import Config
from mmengine.runner import Runner
from mmengine.runner.checkpoint import load_checkpoint

# ====== Paths ======
CONFIG = r'C:/Users/heheh/mmdetection/configs/panoptic_fpn/panoptic-fpn_r50_fpn_1x_coco.py'
CHECKPOINT = r'C:/Users/heheh/mmdetection/checkpoints/panoptic_fpn_r50_fpn_1x_coco_20210821_101153-9668fd13.pth'
DATA_ROOT = r'C:/Users/heheh/mmdetection/data/coco'
PAN_JSON  = os.path.join(DATA_ROOT, 'annotations', 'panoptic_val2017.json')
IMG_DIR   = os.path.join(DATA_ROOT, 'val2017')
PAN_SEG   = os.path.join(DATA_ROOT, 'annotations', 'panoptic_val2017')  # PNG dir
ADV_SAVE_DIR = os.path.join(DATA_ROOT, 'panoptic_fpn_adv')
os.makedirs(ADV_SAVE_DIR, exist_ok=True)

# ====== Attack hyperparams (pixel-space) ======
epsilon   = 8/255.0           # L_inf radius
alpha     = 2/255.0           # step size
num_steps = 20
random_start = True           # good for stronger attacks

# ====== Build model ======
cfg = Config.fromfile(CONFIG)
cfg.default_scope = 'mmdet'
cfg.load_from = CHECKPOINT
cfg.work_dir = './work_dirs/pgd_panoptic'
if hasattr(cfg, 'visualizer'):
    cfg.visualizer.vis_backends = None

runner = Runner.from_cfg(cfg)
model = runner.model
model.eval()
device = next(model.parameters()).device
# Ensure weights are loaded for ad-hoc usage:
load_checkpoint(model, CHECKPOINT, map_location=device)
model.cfg = cfg  # handy, if you later use APIs that expect .cfg

# ====== Build an ATTACK dataloader WITH GT (test_mode=False) ======
# Use the training pipeline (loads bboxes/masks/sem seg) but point to VAL panoptic
attack_dl_cfg = copy.deepcopy(cfg.train_dataloader)
ds = attack_dl_cfg.dataset
# If your train_dataloader wraps the dataset (e.g., RepeatDataset), unwrap to the innermost:
while hasattr(ds, 'dataset'):
    ds = ds.dataset
ds.type = 'CocoPanopticDataset'
ds.data_root = DATA_ROOT
ds.ann_file = PAN_JSON
ds.data_prefix = dict(img=IMG_DIR, seg=PAN_SEG)  # img dir + GT panoptic PNG dir
ds.test_mode = False  # IMPORTANT: include ground-truth for losses

# Keep batch size 1 (simplifies per-image saving & metadata handling)
attack_dl_cfg.batch_size = 1
attack_dl_cfg.num_workers = 2

attack_loader = runner.build_dataloader(attack_dl_cfg)

# ====== Normalization bounds (model uses mean/std in 0-255 space) ======
pre = model.data_preprocessor
mean = torch.tensor(pre.mean, device=device).view(1,3,1,1)
std  = torch.tensor(pre.std,  device=device).view(1,3,1,1)
lower = (0.0   - mean) / std
upper = (255.0 - mean) / std

eps_norm   = (epsilon * 255.0) / std
alpha_norm = (alpha   * 255.0) / std

# ====== Utilities ======
def sum_all_losses(losses: dict):
    total = 0.0
    for v in losses.values():
        if isinstance(v, dict):
            for t in v.values():
                if torch.is_tensor(t):
                    total = total + t.sum()
        elif torch.is_tensor(v):
            total = total + v.sum()
    return total

@torch.no_grad()
def save_adv_in_original_size(norm_img_1CHW, data_sample, save_path):
    """Denorm -> unpad -> resize back to ori size -> save"""
    x = norm_img_1CHW.clone() * std + mean        # 0..255
    x = x.clamp(0,255)[0].permute(1,2,0).cpu().numpy().astype(np.uint8)  # HxWx3

    img_h, img_w = data_sample.metainfo['img_shape'][:2]   # after resize, before pad
    x = x[:img_h, :img_w, :]                               # remove pad

    ori_h, ori_w = data_sample.metainfo['ori_shape'][:2]
    Image.fromarray(x).resize((ori_w, ori_h), Image.BILINEAR).save(save_path, quality=95)

# ====== PGD loop ======
torch.set_grad_enabled(True)

for data_batch in attack_loader:
    # Normalize & collate like the runner would
    data = model.data_preprocessor(data_batch, training=False)  # {'inputs': tensor, 'data_samples': list}
    imgs = data['inputs']             # (1,3,H,W), normalized
    samples = data['data_samples']    # [DetDataSample] with GT for panoptic (instances + semantic)

    x = imgs.detach().clone().to(device)       # clean normalized
    if random_start:
        x = x + torch.empty_like(x).uniform_(-1,1) * eps_norm
        x = torch.max(torch.min(x, x + eps_norm), x - eps_norm)  # stay in ball
        x = torch.max(torch.min(x, upper), lower)                # valid range

    x.requires_grad_(True)

    # Derive a good filename
    meta = samples[0].metainfo
    base = os.path.splitext(os.path.basename(meta.get('ori_filename', meta.get('img_path', 'img'))))[0]
    save_path = os.path.join(ADV_SAVE_DIR, f'{base}.jpg')

    for t in range(num_steps):
        H, W = x.shape[2], x.shape[3]
        samples[0].set_metainfo({
            'img_shape': (H, W, 3),
            'pad_shape': (H, W, 3),
            'batch_input_shape': (H, W),
        })

        model.zero_grad(set_to_none=True)
        losses = model(x, samples, mode='loss')   # PanopticFPN: RPN/ROI + semantic head losses
        loss = sum_all_losses(losses)
        loss.backward()

        with torch.no_grad():
            grad = x.grad
            x = x + alpha_norm * torch.sign(grad)           # untargeted: maximize total loss
            x = torch.max(torch.min(x, imgs + eps_norm), imgs - eps_norm)  # project to L_inf ball
            x = torch.max(torch.min(x, upper), lower)       # clamp valid range

        x.requires_grad_(True)

    with torch.no_grad():
        save_adv_in_original_size(x, samples[0], save_path)
    print(f'[PGD PanopticFPN] Saved {save_path}')
